In [ ]:
from training import *

Returns the data, split into training, validation and test. 

Choose the number of time steps (events of each samples) by changing the number_time_steps variable.
The version variable and the columns varable is used to pick the features of the events. 

    version = 'start_pos' # only extract the start position. 
   
    version = 'best' # extracts the best features found in experiments
   
    version = 'pick_col' # chooses the columns specified in the columns variable
   
    version = 'remove_col' # removes the columns specified in the columns variable
   

After running the cell the columns actually chosen can be accessed by running dataset.col

In [ ]:
dataset_name = 'fotball'
number_time_steps = 10
version = 'best'
columns = []
resample = False
dataset, (X_train, y_train), (X_val, y_val), (X_test, y_test) = read_data(dataset_name, number_time_steps, version, resample, columns)

Runs a training session. 
Choose the type of network archtecture by changing the network_type variable.
Choose the number of configurations to train by changing the number_combinations variable.
Choose the number of times to train each configuration by changing the number_training_per_combinatination variable.

In [ ]:
network_type = 'lstm' # one in ['mlp', 'conv1', 'conv2', 'lstm', 'conv_lstm']
number_combinations = 10 # number of combinations of hyperparameters to test
number_training_per_combinatination = 3 # number of time to train each combination of parameters

# total number of trainings done is number_combinations * number_training_per_combinatination
hists = run_training(dataset, network_type, number_combinations, X_train, y_train, X_val, y_val, number_training_per_combinatination)
sorted_hists, goal_metric = summarize_set(hists)

Retrains the best configuration. 
Will keep a model with higher metric than during the initial training, or the last out of 10 training runs. 

In [ ]:
# retrain best configuration
retrained_model, retrain_hist = retrain_from_conf(sorted_hists[0]['configuration'], dataset, X_train, y_train, X_val, y_val, goal_metric)
# evaluate best model on test dataset
test_hist = evaluate_model(retrained_model, X_test, y_test, dataset.name) 
# save training history
out = save_session(sorted_hists[0]['configuration'], dataset, retrain_hist, test_hist, save_file=True, filename=None)
# print shot summary
print_retrain_summary(retrain_hist, test_hist, dataset.col, out['dataset']['main_metric'], goal_metric)